In [1]:
import numpy as np
from pathlib import Path
from tqdm import tqdm
import matplotlib.pyplot as plt

from hfnet.datasets.colmap import Colmap
from hfnet.evaluation.loaders import sift_loader, export_loader, fast_loader, harris_loader
from hfnet.evaluation.local_descriptors import evaluate
from hfnet.utils import tools

from utils import plot_images

%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
sequences = ['british_museum', 'florence_cathedral_side', 'lincoln_memorial_statue', 
             'london_bridge', 'milan_cathedral', 'mount_rushmore', 'piazza_san_marco',
             'reichstag', 'sagrada_familia', 'st_pauls_cathedral', 'united_states_capitol']

In [3]:
data_config = {
    'make_pairs': True, 'shuffle': True, 'pair_thresh': 0.4, 'max_num_pairs': 200
}
methods = {
    'sift': {
        'predictor': sift_loader,
        'root': False,
        'num_features': 1000,
        'do_ratio_test': True,
    },
    'lfnet': {
        'experiment': 'lfnet/colmap_all_kpts-1000',
        'predictor': export_loader,
        'num_features': 1000,
        'do_ratio_test': False,
    },
}
eval_config = {
    'correct_match_thresh': 5,
    'correct_trans_thresh': 1,
    'correct_rot_thresh': 1,
}

In [4]:
all_metrics = []
for seq in sequences:
    data_config['sequences'] = seq
    dataset = Colmap(**data_config)
    seq_metrics = {}
    #print(f'--- {seq}')
    for method, config in methods.items():
        config = tools.dict_update(config, eval_config)
        data_iter = dataset.get_test_set()
        metrics, _, _, _ = evaluate(data_iter, config, is_2d=False)
        seq_metrics[method] = metrics
    all_metrics.append(seq_metrics)
    
names = list(metrics.keys())
w = 24
pad = w//len(methods)
s1 = f'{"sequences":^{w}}'
s2 = f'{"":^{w}}'
for n in names:
    s1 += f'{n:^{w}.{w}}'
    for method in methods:
        s2 += f'{method:^{pad}}'
print(s1)
print(s2)
    
for seq, met in zip(sequences, all_metrics):
    s = f'{seq:^{w}.{w}}'
    for n in names:
        for method in methods:
            s += f'{met[method][n]:^{pad}.3f}'
    print(s)

s = f'{"average":^{w}.{w}}'
for n in names:
    for method in methods:
        avg = np.mean([met[method][n] for met in all_metrics])
        s += f'{avg:^{pad}.3f}'
print(s)

[10/14/2018 22:33:00 INFO] Colmap sequence british_museum contains 200 pairs
200it [01:08,  2.64it/s]
200it [00:23, 11.60it/s]
[10/14/2018 22:34:35 INFO] Colmap sequence florence_cathedral_side contains 200 pairs
200it [01:19,  2.87it/s]
200it [00:26, 11.33it/s]
[10/14/2018 22:36:32 INFO] Colmap sequence lincoln_memorial_statue contains 200 pairs
200it [01:17,  3.07it/s]
200it [00:31,  9.43it/s]
[10/14/2018 22:38:30 INFO] Colmap sequence london_bridge contains 200 pairs
200it [01:13,  3.21it/s]
200it [00:28, 10.06it/s]
[10/14/2018 22:40:15 INFO] Colmap sequence milan_cathedral contains 200 pairs
200it [01:14,  3.04it/s]
200it [00:25, 10.39it/s]
[10/14/2018 22:41:59 INFO] Colmap sequence mount_rushmore contains 200 pairs
200it [01:13,  3.09it/s]
200it [00:26, 11.62it/s]
[10/14/2018 22:43:44 INFO] Colmap sequence piazza_san_marco contains 200 pairs
200it [01:21,  2.73it/s]
200it [00:28, 10.41it/s]
[10/14/2018 22:45:36 INFO] Colmap sequence reichstag contains 200 pairs
200it [01:12,  3.48

       sequences         average_num_keypoints        correctness            matching_score               mAP           
                            sift       lfnet        sift       lfnet        sift       lfnet        sift       lfnet    
     british_museum       1000.000    1000.000     0.430       0.445       0.278       0.352       0.322       0.206    
florence_cathedral_side   1000.000    1000.000     0.830       0.910       0.189       0.261       0.264       0.215    
lincoln_memorial_statue   1000.000    1000.000     0.785       0.555       0.178       0.134       0.200       0.030    
     london_bridge        1000.000    1000.000     0.760       0.745       0.168       0.212       0.175       0.071    
    milan_cathedral       1000.000    1000.000     0.540       0.490       0.166       0.244       0.222       0.151    
     mount_rushmore       1000.000    1000.000     0.890       0.835       0.223       0.240       0.352       0.109    
    piazza_san_marco      1000.0

        average           1000.000    1000.000     0.687       0.644       0.180       0.208       0.211       0.101    
